In [1]:
from datetime import datetime, timedelta

In [12]:
start = datetime.strptime("21-06-2014", "%d-%m-%Y")

In [3]:
end = datetime.strptime("07-07-2014", "%d-%m-%Y")

In [7]:
date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days)]

In [13]:
date_range = []

In [14]:
for date in date_generated:
    date_range.append(date.strftime("%Y%m%d"))

In [15]:
date_range

['20140621',
 '20140622',
 '20140623',
 '20140624',
 '20140625',
 '20140626',
 '20140627',
 '20140628',
 '20140629',
 '20140630',
 '20140701',
 '20140702',
 '20140703',
 '20140704',
 '20140705',
 '20140706']

In [7]:
from datetime import datetime, timedelta
 
start = datetime.strptime("01-01-2018", "%d-%m-%Y")
end = datetime.strptime("04-01-2018", "%d-%m-%Y")
date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days + 1)]
date_range = []

for date in date_generated:
    date_range.append(date.strftime("%Y-%m-%d"))

print(date_range)

['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04']


In [8]:
date_range

['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04']

In [9]:
from datetime import datetime, timedelta
 
start = datetime.strptime("01-01-2018", "%d-%m-%Y")
end = datetime.strptime("02-01-2018", "%d-%m-%Y")
date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days + 1)]
date_range = []

for date in date_generated:
    date_range.append(date.strftime("%Y-%m-%d"))

print(date_range)

['2018-01-01', '2018-01-02']


In [4]:
from datetime import datetime, timedelta
 
start = datetime.strptime("01-01-2018", "%d-%m-%Y")
end = datetime.strptime("05-01-2018", "%d-%m-%Y")
date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days + 1)]
date_range = []

for date in date_generated:
    date_range.append(date.strftime("%Y-%m-%d"))
    print(date_range[:])

#print(date_range)

['2018-01-01']
['2018-01-01', '2018-01-02']
['2018-01-01', '2018-01-02', '2018-01-03']
['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04']
['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05']


In [4]:
from ecmwfapi import ECMWFDataServer
from datetime import datetime, timedelta
 
start = datetime.strptime("01-01-2018", "%d-%m-%Y")
end = datetime.strptime("02-01-2018", "%d-%m-%Y")
date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days + 1)]
date_range = []

for date in date_generated:
    date_range.append(date.strftime("%Y-%m-%d"))

for date in date_generated:
    print str(date.strftime("%Y%m%d"))
    print str(date)

    
for date1 in date_range:
    server = ECMWFDataServer()
    server.retrieve({
        "class"  : "ei",
        "dataset": "interim",
        "date"   : str(date.strftime("%Y-%m-%d")),
        "expver" : "1",
        "grid"   : "0.75/0.75",
        "levtype": "sfc",
        "param"  : "228.128",
        "step"   : "12",
        "stream" : "oper",
        "time"   : "00:00:00/12:00:00",
        "type"   : "fc",
        "format" : "netcdf",
        "target" : "tp_" + date.strftime("%Y%m%d") + ".nc",
    })

20180101
2018-01-01 00:00:00
20180102
2018-01-02 00:00:00
2019-12-02 08:44:17 ECMWF API python library 1.5.4
2019-12-02 08:44:17 ECMWF API at https://api.ecmwf.int/v1
2019-12-02 08:44:19 Welcome Juan Manuel Cotrino Palma
2019-12-02 08:44:20 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2019-12-02 08:44:21 Request submitted
2019-12-02 08:44:21 Request id: 5de51535cad519329a16dbcb
2019-12-02 08:44:21 Request is submitted
2019-12-02 08:44:23 Request is queued


KeyboardInterrupt: 

In [3]:
import time, sys
from datetime import datetime, timedelta

from netCDF4 import Dataset, date2num, num2date
import numpy as np
 
day = 20180101
f_in = 'tp_%d.nc' % day
f_out = 'daily-tp_%d.nc' % day
 
d = datetime.strptime(str(day), '%Y%m%d')
time_needed = [d + timedelta(hours = 12), d + timedelta(days = 1)]
 
with Dataset(f_in) as ds_src:
    var_time = ds_src.variables['time']
    time_avail = num2date(var_time[:], var_time.units,
            calendar = var_time.calendar)
 
    indices = []
    for tm in time_needed:
        a = np.where(time_avail == tm)[0]
        if len(a) == 0:
            sys.stderr.write('Error: precipitation data is missing/incomplete - %s!\n'
                    % tm.strftime('%Y%m%d %H:%M:%S'))
            sys.exit(200)
        else:
            print('Found %s' % tm.strftime('%Y%m%d %H:%M:%S'))
            indices.append(a[0])
 
    var_tp = ds_src.variables['tp']
    tp_values_set = False
    for idx in indices:
        if not tp_values_set:
            data = var_tp[idx, :, :]
            tp_values_set = True
        else:
            data += var_tp[idx, :, :]
         
    with Dataset(f_out, mode = 'w', format = 'NETCDF3_64BIT_OFFSET') as ds_dest:
        # Dimensions
        for name in ['latitude', 'longitude']:
            dim_src = ds_src.dimensions[name]
            ds_dest.createDimension(name, dim_src.size)
            var_src = ds_src.variables[name]
            var_dest = ds_dest.createVariable(name, var_src.datatype, (name,))
            var_dest[:] = var_src[:]
            var_dest.setncattr('units', var_src.units)
            var_dest.setncattr('long_name', var_src.long_name)
 
        ds_dest.createDimension('time', None)
 
        # Variables
        var = ds_dest.createVariable('time', np.int32, ('time',))
        time_units = 'hours since 1900-01-01 00:00:00'
        time_cal = 'gregorian'
        var[:] = date2num([d], units = time_units, calendar = time_cal)
        var.setncattr('units', time_units)
        var.setncattr('long_name', 'time')
        var.setncattr('calendar', time_cal)
        var = ds_dest.createVariable(var_tp.name, np.double, var_tp.dimensions)
        var[0, :, :] = data
        var.setncattr('units', var_tp.units)
        var.setncattr('long_name', var_tp.long_name)
 
        # Attributes
        ds_dest.setncattr('Conventions', 'CF-1.6')
        ds_dest.setncattr('history', '%s %s'
                % (datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                ' '.join(time.tzname)))
 
        print('Done! Daily total precipitation saved in %s' % f_out)

ImportError: cannot import name __pdoc__